In [5]:
import pandas as pd
import pydicom
import os
from tqdm import tqdm
from glob import glob
import nibabel as nib
import numpy as np
# from drr_raysum import *
import pickle
import scipy.ndimage

In [13]:
AVIEW_PATH = '../data/AVIEW_lungseg/01'
df_aview = pd.read_csv('../preprocess/nii_1018_aview.csv')

In [7]:
def resample(image, slices, new_spacing=[1,1,1]):
    '''
    3d resize
    '''
    # Determine current pixel spacing
    # SliceThickness를 쓰면 overlapping되게 찍은 경우 고려가 안되어서 ImagePositionPatient을 사용
    spacing = np.array(list(slices[0].PixelSpacing) + [np.abs(slices[1].ImagePositionPatient[-1]-slices[0].ImagePositionPatient[-1])], dtype=np.float32)
    
    resize_factor = spacing / new_spacing
    new_shape = np.round(image.shape * resize_factor)
    
    # recompute the resize factor and spacing such that we match the rounded new shape above
    rounded_resize_factor = new_shape / image.shape
    rounded_new_spacing = spacing / rounded_resize_factor
    
    # zoom with resize factor
    image = scipy.ndimage.interpolation.zoom(image, rounded_resize_factor, mode='nearest')
    
    return image, rounded_new_spacing

def load_mask_dict(mask_path):
    segment_arr_dict = dict()
    mask_name = [x.split('.')[-3] for x in mask_path]
    for obj in range(len(mask_name)):
        proxy = nib.load(mask_path[obj])
        header = proxy.header
        arr = proxy.get_fdata()
        img_shape = arr.shape
        
        roi_img3d = np.zeros(img_shape)
        for i in range(img_shape[-1]):
            roi_img3d[:, :, i] = arr[:, :, ::-1][:, :, i].T

        segment_arr_dict[mask_name[obj]] = roi_img3d
        
    return segment_arr_dict, mask_name

### lung pickle 생성

In [ ]:
empty_list = []
for idx in tqdm(range(len(df_aview))):
    lung_dict_L={}
    lung_dict_R={}
    
    pickle_name = '.'.join(df_aview.iloc[idx]['path'].split('/')[-1].split('.')[:2])
    lung_path = df_aview.iloc[idx]['aview_dir']
    patient_mask_folder = glob(os.path.join(AVIEW_PATH,lung_path))[0]
    
    mask_path = sorted(glob(f"{patient_mask_folder}/stor/objects/*.nii.gz"))
    dcm_path = sorted(glob(f"{patient_mask_folder}/*dcm"))
    
    # empty
    if (len(mask_path) == 0) | (len(dcm_path) == 0):   
        empty_list.append(pickle_name)
        continue
    
    slices = []
    for fname in dcm_path:
        ds = pydicom.dcmread(fname, force=True)
        slices.append(ds)
        slices = sorted(slices, key=lambda s: s[0x020, 0x032][-1], reverse=True)
    
    segment_arr_dict, mask_name = load_mask_dict(mask_path)
    segment_arr_dict['LtLung'], _ = resample(segment_arr_dict['LtLung'], slices)
    segment_arr_dict['RtLung'], _ = resample(segment_arr_dict['RtLung'], slices)
    
    LtLung = np.where(segment_arr_dict['LtLung'] > 2)
    RtLung = np.where(segment_arr_dict['RtLung'] > 2)
    
    # (x, y, z)
    LtLung_ = list(zip(LtLung[0].tolist(), LtLung[1].tolist(), LtLung[2].tolist()))
    RtLung_ = list(zip(RtLung[0].tolist(), RtLung[1].tolist(), RtLung[2].tolist()))
    
    lung_dict_L['Lung']=np.array(LtLung_)
    lung_dict_R['Lung']=np.array(RtLung_)
    
    # save pickle
    with open(f'../PICKLE_lungLR/{pickle_name}_Left.pickle', 'wb') as f:
        pickle.dump(lung_dict_L, f, pickle.HIGHEST_PROTOCOL)
    with open(f'../PICKLE_lungLR/{pickle_name}_Right.pickle', 'wb') as f:
        pickle.dump(lung_dict_R, f, pickle.HIGHEST_PROTOCOL)

### lung pickle -> lung npy

In [ ]:
# make .pickle file to .npy
for pickle_idx in tqdm(range(len_pickle)):
    try:
        img = nib.load(img_list[pickle_idx]).get_fdata()
        fname = '_'.join(pickle_list[pickle_idx].split('/')[-1].split('_')[:-1])

        with open(f'../PICKLE_lungLR/{fname}_Right.pickle', 'rb') as p:
            pickle_lungR = pickle.load(p)
        lung_R = pickle_lungR['Lung']

        with open(f'../PICKLE_lungLR/{fname}_Left.pickle', 'rb') as p:
            pickle_lungL = pickle.load(p)
        lung_L = pickle_lungL['Lung']

        lung_npy_R = np.zeros(img.shape, dtype=np.int64)
        lung_npy_L = np.zeros(img.shape, dtype=np.int64)

        for idx_L in range(len(lung_L)):
            lung_npy_L[lung_L[idx_L][0], lung_L[idx_L][1], lung_L[idx_L][2]] = 255

        for idx_R in range(len(lung_R)):
            lung_npy_R[lung_R[idx_R][0], lung_R[idx_R][1], lung_R[idx_R][2]] = 255

        np.save(f'../npy_lungLR/{fname}_Left', lung_npy_L)
        np.save(f'../npy_lungLR/{fname}_Right', lung_npy_R)
    except:
        print(fname)
    